# This is just a small sample analysis of the Steam data we've collected and processed

### Table of contents

* Importing data

* Checking tables

* Analysis: Are Sci-fi RPGs more popular than Fantasy ones?

    * Task
    * Reorganizing data
    * Making plots
    * Analyzing the results


In [1]:
# Module imports
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time
import re
import ast
import itertools

# third-party imports
import numpy as np
import seaborn as sns
import pandas as pd

In [2]:
# Loading data tables
steam = pd.read_csv('../data/export/steam.csv')
missing_ids = pd.read_csv('../data/export/missing_ids.csv')
steam_description_data = pd.read_csv('../data/export/steam_description_data.csv')
steam_media_data = pd.read_csv('../data/export/steam_media_data.csv')
steam_optional = pd.read_csv('../data/export/steam_optional.csv')
steam_packages_info = pd.read_csv('../data/export/steam_packages_info.csv')
steam_requirements_data = pd.read_csv('../data/export/steam_requirements_data.csv')
steam_support_info = pd.read_csv('../data/export/steam_support_info.csv')
steamspy_tag_data = pd.read_csv('../data/export/steamspy_tag_data.csv')

## Let's check our tables:

In [3]:
steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102504 entries, 0 to 102503
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   appid                102504 non-null  int64  
 1   type                 102504 non-null  object 
 2   name                 102504 non-null  object 
 3   required_age         102504 non-null  int64  
 4   dlc                  9696 non-null    object 
 5   fullgame             34607 non-null   object 
 6   supported_languages  102352 non-null  object 
 7   developers           102463 non-null  object 
 8   publishers           102464 non-null  object 
 9   packages             81153 non-null   object 
 10  platforms            102504 non-null  object 
 11  categories           102398 non-null  object 
 12  genres               102311 non-null  object 
 13  achievements         102504 non-null  float64
 14  release_date         95676 non-null   object 
 15  supported_audio  

In [4]:
steam.sample(10)

,appid,type,name,required_age,dlc,fullgame,supported_languages,developers,publishers,packages,...,coming_soon,price,review_score,total_positive,total_negative,rating,owners,average_forever,median_forever,tags
10427,376969,dlc,Train Simulator: DB BR 18 Steam Loco Add-On,0,NaN,"{'appid': '24010', 'name': 'Train Simulator Cl...","['English', 'German']",['Eisenbahnwerk'],['Dovetail Games - Trains'],[69916],...,False,13.99,7.0,12.0,2.0,69.908968,"0 .. 20,000",0.0,0.0,['Simulation']
83794,1646790,game,Striving for Light,0,[1829720],NaN,"['English', 'French', 'German', 'Italian', 'Sp...",['Igniting Spark Games'],['Igniting Spark Games'],[585161],...,False,6.59,0.0,6.0,1.0,66.616422,"0 .. 20,000",0.0,0.0,"['Indie', 'Action Roguelike', 'Hack and Slash'..."
25349,645500,game,Cat Sorter VR,0,NaN,NaN,"['English', 'Simplified Chinese']",['Pawmigo Games'],['Pawmigo Games'],"[181198, 210502]",...,False,7.99,7.0,20.0,5.0,68.749500,"0 .. 20,000",0.0,0.0,"['Casual', 'Indie', 'Simulation', 'VR', 'Cats']"
22918,600370,game,Paradigm,0,[626180],NaN,['English'],['Jacob Janerka'],['Jacob Janerka'],[157681],...,False,14.99,9.0,780.0,29.0,90.233453,"100,000 .. 200,000",166.0,195.0,"['Adventure', 'Indie', 'Point & Click', 'Comed..."
19469,542410,game,True Blades™,0,NaN,NaN,['English'],['Gamerizon Studio Inc.'],['Gamerizon Studio Inc.'],[133449],...,False,22.99,5.0,9.0,11.0,46.999596,"0 .. 20,000",0.0,0.0,"['Adventure', 'Strategy', 'Action', 'RPG', 'Vi..."
26063,658490,game,Drive Isle,0,NaN,NaN,"['English', 'Finnish', 'Spanish - Spain']",['Roadfactory'],['Roadfactory'],[186247],...,False,2.39,5.0,45.0,32.0,56.167271,"0 .. 20,000",0.0,0.0,"['Racing', 'Simulation', 'Early Access', 'Open..."
23657,614000,dlc,X-Plane 11 - Add-on: Aerosoft - Airport Rio de...,0,NaN,"{'appid': '269950', 'name': 'X-Plane 11'}","['English', 'German']",['Globall Art'],['Aerosoft GmbH'],[164486],...,False,29.69,0.0,3.0,3.0,50.000000,"0 .. 20,000",0.0,0.0,['Simulation']
61274,1247620,dlc,Lock 'n Load Tactical Digital: Heroes of Norma...,0,NaN,"{'appid': '1149940', 'name': ""Lock 'n Load Tac...","['English', 'Finnish', 'French', 'German', 'It...","[""Lock 'n Load Publishing""]","[""Lock 'n Load Publishing""]",[432939],...,False,8.19,0.0,5.0,1.0,64.777567,NaN,NaN,NaN,NaN
91110,1756390,game,Mercs Fully Loaded,0,NaN,NaN,['English'],['ByteBomb Studios'],['ByteBomb Studios'],[629883],...,False,16.79,0.0,1.0,1.0,50.000000,"20,000 .. 50,000",0.0,0.0,"['Early Access', 'RPG', 'Action', 'Third-Perso..."
8182,342300,game,Zombie Kill of the Week - Reborn,0,[359540],NaN,['English'],['Still Running'],"['Merge Games', 'Tagstar Games']","[57715, 60850]",...,False,5.99,8.0,584.0,45.0,86.690864,"50,000 .. 100,000",309.0,242.0,"['Action', 'Zombies', 'Indie', '2D', 'Multipla..."


In [5]:
steam[['name','supported_languages']].sample(10)

,name,supported_languages
16336,Hover 2030,['English']
360,Sprint Cars Road to Knoxville,['English']
17819,Duckpocalypse,['English']
21171,AudioBeats,"['English', 'Simplified Chinese']"
42704,Lemnis Gate,"['English', 'French', 'German', 'Italian', 'Ja..."
74513,Hanoi Puzzles: Magnetrix,['English']
54113,The Surge 2 - Future Shock Weapon Pack,"['English', 'French', 'German', 'Italian', 'Ja..."
88943,Click and Slay,['English']
99191,Art of Stunt,"['English', 'Simplified Chinese']"
48953,City Zombies,"['Arabic', 'Bulgarian', 'Czech', 'Danish', 'Du..."


In [10]:
def f(x):
    if pd.isna(x):
        return []
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []
    
temp_df = steam[['name','supported_languages']].copy()
#temp_df['supported_languages'] = temp_df['supported_languages'].fillna("[]")
#temp_df['supported_languages'] = temp_df['supported_languages'].apply(lambda x:f(x))
temp_df['supported_languages'] = temp_df[temp_df['supported_languages'].notnull()]['supported_languages'].apply(eval)
temp_df.explode('supported_languages')

,name,supported_languages
0,Counter-Strike,English
0,Counter-Strike,French
0,Counter-Strike,German
0,Counter-Strike,Italian
0,Counter-Strike,Korean
...,...,...
102503,Bioshock Infinite: Columbia's Finest,Japanese
102503,Bioshock Infinite: Columbia's Finest,Polish
102503,Bioshock Infinite: Columbia's Finest,Portuguese - Brazil
102503,Bioshock Infinite: Columbia's Finest,Russian


In [11]:
missing_ids.sample(10)

,appid,reason
153,NaN,Missing app name
380,NaN,Missing app name
276,NaN,Missing app name
594,NaN,Missing app name
177,NaN,Missing app name
340,NaN,Missing app name
327,NaN,Missing app name
34,NaN,Missing app name
47,NaN,Missing app name
642,NaN,Missing app name


In [20]:
steam_description_data.sample(10)

,appid,detailed_description,about_the_game,short_description
96677,1862230,PuckBash is the next evolution of traditional ...,PuckBash is the next evolution of traditional ...,Experience the frenzy of Air Hockey pushed to ...
93071,1797470,Give your Basement a spooky new look! The new ...,Give your Basement a spooky new look! The new ...,Give your Basement a spooky new look with the ...
53912,1135190,Hoppa is a short adventure game about a baby d...,Hoppa is a short adventure game about a baby d...,"A tiny adventure game, pole vault over dangero..."
26263,664360,A fictional route based in the United States o...,A fictional route based in the United States o...,A fictional route in the USA where the Burling...
82897,1635210,"<h1>ADD TO WISHLIST</h1><p><a href=""https://st...","Broken Blades is a platform, 2D action game wi...","Avoid traps, upgrade your sword and reach powe..."
33697,794170,Purchase this DLC disables score limit on this...,Purchase this DLC disables score limit on this...,Locomotion 2018 Table is a modernized version ...
92064,1779280,<strong>Humankind in a nutshell is a 4X strate...,<strong>Humankind in a nutshell is a 4X strate...,Humankind in a nutshell is a 4X strategy game ...
93769,1806580,"<ul class=""bb_ul""><li> 5 universes to explore<...","<ul class=""bb_ul""><li> 5 universes to explore<...",Neon Bricks is a arcade style game where you c...
43207,959350,"<h2 class=""bb_tag"">[disclaimer]</h2>All charac...","<h2 class=""bb_tag"">[disclaimer]</h2>All charac...",Sengoku Neet is Strategy Conquest Simulation E...
14842,456560,The original soundtrack to Bezier. 22 tracks s...,The original soundtrack to Bezier. 22 tracks s...,The original soundtrack to Bezier. Provided in...


In [21]:
steam_media_data.sample(10)

,appid,header_image,screenshots,background,movies
65162,1319850,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 256881921, 'name': 'ValiDate: Struggli..."
96528,1859140,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 256877364, 'name': ""Pirate's Solitaire..."
91873,1775900,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN
56254,1172210,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN
12199,408550,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN
98668,1906180,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 256880955, 'name': 'Memory Traces Japa..."
7301,327880,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 256690085, 'name': 'Redux Launch Trail..."
48572,1052480,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 256773221, 'name': 'Trailer FR 2020', ..."
62256,1266450,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 256778163, 'name': '予告編', 'thumbnail':..."
98800,1909250,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN


In [22]:
steam_optional.sample(10)

,appid,drm_notice,ext_user_account_notice,demos,content_descriptors,metacritic_score,metacritic_url
22100,1922060,NaN,NaN,NaN,There are weapons in the game like knives and ...,NaN,NaN
9116,1099293,NaN,NaN,NaN,This Game may contain content not appropriate ...,NaN,NaN
18036,1626740,NaN,NaN,NaN,This game contains adult content that may not ...,NaN,NaN
10745,1181880,NaN,NaN,[1222590],NaN,NaN,NaN
19650,1740230,NaN,NaN,[1797920],NaN,NaN,NaN
3576,521540,NaN,NaN,[530840],NaN,NaN,NaN
22244,1934700,NaN,NaN,[1945830],NaN,NaN,NaN
12174,1248370,NaN,NaN,[1248410],NaN,NaN,NaN
3965,581380,NaN,NaN,[1262250],NaN,NaN,NaN
9559,1122150,NaN,NaN,NaN,"""ELEA"" contains some brief sexual suggestive m...",NaN,NaN


In [23]:
steam_packages_info.sample(10)

,appid,type,title,is_recurring_subscription,subs
34266,871280,default,Buy Grisaia Phantom Trigger Vol.5,False,"[{'packageid': 275341, 'percent_savings_text':..."
16822,517450,default,Buy Hue Official Soundtrack,False,"[{'packageid': 121089, 'percent_savings_text':..."
47604,1141380,default,Buy Ladies Orders,False,"[{'packageid': 388433, 'percent_savings_text':..."
66409,1570420,default,Buy Clan of Death,False,"[{'packageid': 555018, 'percent_savings_text':..."
62066,1463080,default,Buy Snow Survival,False,"[{'packageid': 515136, 'percent_savings_text':..."
2616,222586,default,Buy Train Simulator: BR Class 87 Loco Add-On,False,"[{'packageid': 28657, 'percent_savings_text': ..."
27043,722380,default,Buy Blindsight,False,"[{'packageid': 209803, 'percent_savings_text':..."
4759,282030,default,Buy Carmageddon 2: Carpocalypse Now,False,"[{'packageid': 38766, 'percent_savings_text': ..."
5715,301245,default,Buy Fantasy Grounds - 4E: Lands of Darkness #2...,False,"[{'packageid': 44137, 'percent_savings_text': ..."
78498,1862600,default,Buy Dino Island Adventure,False,"[{'packageid': 670167, 'percent_savings_text':..."


In [24]:
steam_requirements_data.sample(10)

,appid,pc_requirements,mac_requirements,linux_requirements,pc_minimum,pc_recommended,mac_minimum,mac_recommended
36712,1712220,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],Memory: 1 GB RAM Storage: 300 MB available space,NaN,Memory: 1 GB RAM Storage: 400 MB available space,NaN
36422,1699340,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],Additional Notes: System Supporting RPG Maker MZ,NaN,Additional Notes: System Supporting RPG Maker MZ,NaN
396,94202,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...","{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],OS: Microsoft® Windows® XP with SP2 / Vista / ...,NaN,OS: OS X version Snow Leopard 10.6.3 or later....,OS: OS X version Yosemite 10.10.3 or later. Pr...
33098,1556200,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating system,Requires a 64-bit processor and operating system
19930,1044090,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating system,Requires a 64-bit processor and operating system
15716,874690,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,OS: Windows XP SP2+ Processor: SSE2 instructio...,NaN,OS: Mac OS X 10.8+ Processor: SSE2 instruction...,NaN
42406,31220,{'minimum': '<ul>\n\t\t\t\t\t\t\t\t\t<li><stro...,{'minimum': '<ul>\n\t\t\t\t\t\t\t\t\t<li><stro...,[],OS: Windows XP / Vista / Windows 7Processor: 2...,NaN,OS: Mac OS X 10.5 or laterProcessor: Intel Cor...,NaN
10820,676700,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,OS: Windows XP SP2+ Processor: Dual Core at 2....,NaN,OS: Mac OS X 10.9+ Processor: Dual Core at 2.8...,NaN
6758,508650,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],OS: Windows 7 or later Processor: 2 GHz Proces...,NaN,OS: OSX 10.9+ Storage: 1024 MB available space,NaN
30792,1460540,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating system,Requires a 64-bit processor and operating syst...,Requires a 64-bit processor and operating system


In [25]:
steam_support_info.sample(10)

,appid,website,support_url,support_email
98128,1903850,NaN,www.unique-games.info,NaN
94687,1831600,http://www.exosyphen.com,http://www.exosyphen.com,contact@exosyphenstudios.com
16277,492972,http://rocksmith.ubi.com/,https://support.ubi.com/,NaN
62634,1279370,NaN,NaN,support@city-connection.co.jp
77463,1546550,NaN,NaN,fallinglightgames@gmail.com
80200,1591730,NaN,https://olivierderop.com/,NaN
41916,944610,NaN,NaN,jeffreyjmarsh@yahoo.com
82239,1631210,NaN,NaN,digitaldreamgames518@gmail.com
80529,1597400,NaN,NaN,kimidori.soft@gmail.com
59704,1229830,http://www.rpgmakerweb.com/,http://www.rpgmakerweb.com/support,NaN


In [19]:
steamspy_tag_data.sample(10)

,appid,1980s,1990s,2.5d,2d,2d_fighter,2d_platformer,360_video,3d,3d_fighter,...,web_publishing,well_written,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
34311,803050,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
94043,1808870,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69966,1404450,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64443,1301990,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75413,1498030,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66425,1343320,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26361,664430,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,35,0,0
74834,1487790,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55469,1158800,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54197,1137801,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Analysis: Are Sci-fi RPGs more popular than Fantasy ones?

## Task

* Determine how popular are Sci-Fi RPGs comparing to the Fantasy ones
    * Number of owners and number of games
    * Total number of tags and tags present
    * Real-time vs Turn-based
    * Changes over time

## Reorganizing data

### We'll need this data:

#### From steam:
* appid
* name
* type: game
* genres: RPG
* release_date
* coming_soon: False
* rating
* total_positive+total_negative
* owners: get average
* tags: filter by "Fantasy" "Sci-fi" "Cyberpunk"

#### From steamspy_tag_data:
* appid
* tags

In [ ]:
# setting indexes
steam = steam.set_index("appid")
steamspy_tag_data = steamspy_tag_data.set_index("appid")

In [32]:
# filtering out the unneeded data part 1
analysis_df = steam[(steam["type"] == "game") 
                    & (steam["coming_soon"] == False)][["name",
                                                        "genres",
                                                        "release_date",
                                                        "rating","total_positive",
                                                        "total_negative",
                                                        "owners",
                                                        "tags"]].copy()
# unwrapping strings -> list
analysis_df['genres'] = analysis_df[analysis_df['genres'].notnull()]['genres'].apply(eval)
analysis_df['tags'] = analysis_df[analysis_df['tags'].notnull()]['tags'].apply(eval)
# setting some columns format
# total reviews
analysis_df['total_reviews'] = analysis_df['total_positive'] + analysis_df['total_negative']

def parse_ownders(owners_str):
    owners_split = owners_str.str.split(' .. ')
    result = 
    return result
# average owners
analysis_df["owners"] = analysis_df["owners"].apply(lambda x: (int(x[0]) + int(x[1])) // 2)

In [33]:
analysis_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55482 entries, 0 to 101840
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            55482 non-null  object 
 1   genres          55387 non-null  object 
 2   release_date    55427 non-null  object 
 3   rating          55482 non-null  float64
 4   total_positive  55482 non-null  float64
 5   total_negative  55482 non-null  float64
 6   owners          50063 non-null  object 
 7   tags            50063 non-null  object 
 8   total_reviews   55482 non-null  float64
dtypes: float64(4), object(5)
memory usage: 4.2+ MB


In [35]:
analysis_df['owners'] = analysis_df['owners'].str.replace(',', '', regex=True).str.replace(' .. ', '-', regex=True)

In [36]:
analysis_df.sample(15)

,name,genres,release_date,rating,total_positive,total_negative,owners,tags,total_reviews
26959,Rockets are Super Hard,[Indie],2020-05-19,75.000000,9.0,0.0,0-20000,"[Space, Science, Difficult, Board Game, Team-B...",9.0
50948,DestructoPod,"[Action, Indie, Early Access]",2019-07-25,59.416365,1.0,0.0,0-20000,"[Action, Indie, Early Access, Destruction, Fli...",1.0
30250,Sudoku Universe / 数独宇宙,"[Casual, Indie, Strategy]",2017-12-18,90.608648,1587.0,75.0,50000-100000,"[Casual, Indie, Strategy, Puzzle, Relaxing, Mi...",1662.0
16094,Circuitous ®,"[Action, Indie]",2019-07-09,62.097187,6.0,2.0,0-20000,"[Action, Indie]",8.0
80105,Rune Factory 4 Special,[RPG],2021-12-07,84.308032,701.0,81.0,20000-50000,"[RPG, Farming Sim, Adventure, Romance, Fantasy...",782.0
73978,Hot Hentai Puzzle Vol.1,"[Casual, Indie]",2020-12-01,73.191159,29.0,5.0,NaN,NaN,34.0
32064,Lumberjack Simulator,[Simulation],2021-10-03,64.007291,86.0,40.0,0-20000,"[Farming Sim, Exploration, Agriculture, Drivin...",126.0
60726,STAR WARS™ Battlefront™ II,"[Action, Adventure]",2020-06-11,85.232896,26211.0,3956.0,1000000-2000000,"[Multiplayer, Singleplayer, FPS, Shooter, Thir...",30167.0
16935,Puzzle Nebula,"[Casual, Indie]",2016-07-05,59.416365,1.0,0.0,20000-50000,"[Indie, Casual, Puzzle]",1.0
24737,Ski Sniper,"[Action, Casual, Sports]",2017-05-18,77.171699,160.0,30.0,20000-50000,"[Casual, Sports, Action, Sniper]",190.0


## Making plots


## Analyzing the results